## Install Dependencies

In [2]:
#r "nuget: CsvHelper"
#r "nuget: Microsoft.Data.Analysis"

Installed Packages CsvHelper, 33.0.1 Microsoft.Data.Analysis, 0.22.1

## Imports && Define Classes

In [3]:
using System;
using System.Collections.Generic;
using System.Globalization;
using System.IO;
using System.Linq;
using CsvHelper;
using CsvHelper.Configuration;
using Microsoft.Data.Analysis;
// Define a class to represent a row in the CSV file
public class PlayRecord
{
    public string PLAY_ID { get; set; }
    public int SONG_ID { get; set; }
    public int CLIENT_ID { get; set; }
    public DateTime PLAY_TS { get; set; }
}

// Define a class to represent the output result
public class DistinctPlayCountResult
{
    public int DISTINCT_PLAY_COUNT { get; set; }
    public int CLIENT_COUNT { get; set; }
}

## read csv Data

In [4]:
// Read the CSV file
var csvFilePath = "./exhibitA-input.csv"; 
var reader = new StreamReader(csvFilePath);
var csv = new CsvReader(reader, new CsvConfiguration(CultureInfo.InvariantCulture)
{
    Delimiter = "\t"
});

// Parse the CSV file into a list of PlayRecord objects
var records = csv.GetRecords<PlayRecord>().ToList();

In [5]:
Console.WriteLine($"Read {records.Count} records from the CSV file.");

Read 1400000 records from the CSV file.


## Make it DataFrame

In [6]:
var playidcol = new StringDataFrameColumn("PLAY_ID");
var songidcol = new PrimitiveDataFrameColumn<int>("SONG_ID");
var clientidcol = new PrimitiveDataFrameColumn<int>("CLIENT_ID");
var playtscol = new PrimitiveDataFrameColumn<DateTime>("PLAY_TS");

foreach (var record in records)
{
    playidcol.Append(record.PLAY_ID);
    songidcol.Append(record.SONG_ID);
    clientidcol.Append(record.CLIENT_ID);
    playtscol.Append(record.PLAY_TS);
}
var DataFrame = new DataFrame(playidcol, songidcol, clientidcol, playtscol);
DataFrame.Head(5).Display();

index,PLAY_ID,SONG_ID,CLIENT_ID,PLAY_TS
0,44BB190BC2493964E053CF0A000AB546,6164,249,2016-09-08 09:16:34Z
1,44BB190BC24A3964E053CF0A000AB546,544,86,2016-10-08 13:54:52Z
2,44BB190BC24B3964E053CF0A000AB546,9648,589,2016-08-08 06:08:53Z
3,44BB190BC24C3964E053CF0A000AB546,7565,656,2016-11-08 17:30:41Z
4,44BB190BC24D3964E053CF0A000AB546,8995,348,2016-11-08 02:40:39Z


In [7]:
DataFrame.Description().Display();

index,Description,SONG_ID,CLIENT_ID,PLAY_TS
0,Length (excluding null values),1400000,1400000,1400000
1,Max,9999,999,<null>
2,Min,1,1,<null>
3,Mean,-1136.7673,499.93915,<null>


## PlayCount and client count analysis

In [8]:
var targetDate = new DateTime(2016, 10, 8);
var filteredRecords = records
    .Where(r => r.PLAY_TS.Date == targetDate)
    .ToList();

// Group by CLIENT_ID and count distinct SONG_ID for each client
var clientDistinctSongCounts = filteredRecords
    .GroupBy(r => r.CLIENT_ID)
    .Select(g => new
    {
        CLIENT_ID = g.Key,
        DISTINCT_SONG_COUNT = g.Select(r => r.SONG_ID).Distinct().Count()
    })
    .ToList();

// Group by DISTINCT_SONG_COUNT and count the number of clients
var result = clientDistinctSongCounts
    .GroupBy(c => c.DISTINCT_SONG_COUNT)
    .Select(g => new
    {
        DISTINCT_PLAY_COUNT = g.Key,
        CLIENT_COUNT = g.Count()
    })
    .OrderBy(r => r.DISTINCT_PLAY_COUNT)
    .ToList();

// Create a DataFrame
var distinctPlayCountColumn = new PrimitiveDataFrameColumn<int>("DISTINCT_PLAY_COUNT");
var clientCountColumn = new PrimitiveDataFrameColumn<int>("CLIENT_COUNT");

foreach (var item in result)
{
    distinctPlayCountColumn.Append(item.DISTINCT_PLAY_COUNT);
    clientCountColumn.Append(item.CLIENT_COUNT);
}

var dataFrame = new DataFrame(distinctPlayCountColumn, clientCountColumn);

dataFrame.Display();

index,DISTINCT_PLAY_COUNT,CLIENT_COUNT
0,281,1
1,293,1
2,298,1
3,299,1
4,300,2
5,301,2
6,302,2
7,303,2
8,304,1
9,305,5


## Save as csv

In [9]:
public static void SaveDataFrameToCsv(DataFrame dataFrame, string filePath, char delimiter)
{
    using var writer = new StreamWriter(filePath);

    // Write the header
    var header = string.Join(delimiter, dataFrame.Columns.Select(c => c.Name));
    writer.WriteLine(header);

    // Write the rows
    for (long i = 0; i < dataFrame.Rows.Count; i++)
    {
        var row = dataFrame.Rows[i];
        var rowValues = new List<string>();

        for (int j = 0; j < dataFrame.Columns.Count; j++)
        {
            rowValues.Add(row[j]?.ToString() ?? string.Empty);
        }

        var line = string.Join(delimiter, rowValues);
        writer.WriteLine(line);
    }
}

In [10]:
// Save the DataFrame to a tab-delimited CSV file
var outputFilePath = "./output.csv";
SaveDataFrameToCsv(dataFrame, outputFilePath, '\t'); // Use tab as the delimiter

Console.WriteLine($"\nDataFrame saved to {outputFilePath}");


DataFrame saved to ./output.csv
